# Notebook to explore lat/long position of wells & find nearest neighbors
### and potentially functions to make a dataset of nearest neighbors within certain limits for each well
### and potentially functions to make a dataset of tops and other characteristics for neighboring wells.

In [1]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
import welly
from welly import Well
import lasio
import glob
from sklearn import neighbors
import pickle
welly.__version__

'0.3.0'

In [2]:
from ipyleaflet import *

In [3]:
%%timeit
import os
env = %env

102 µs ± 2.08 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [4]:
pd.set_option('display.max_rows', 2000)

In [5]:
picks_dic = pd.read_csv('../../SPE_006_originalData/OilSandsDB/PICKS_DIC.TXT',delimiter='\t')
picks = pd.read_csv('../../SPE_006_originalData/OilSandsDB/PICKS.TXT',delimiter='\t')
wells = pd.read_csv('../../SPE_006_originalData/OilSandsDB/WELLS.TXT',delimiter='\t')
gis = pd.read_csv('../../well_lat_lng.csv')
picks_new=picks[picks['HorID']==13000]
picks_paleoz=picks[picks['HorID']==14000]
df_new = pd.merge(wells, picks_new, on='SitID')
df_paleoz = pd.merge(wells, picks_paleoz, on='SitID')
#### NOTE: This now includes the GIS or well_lat_lng dataset too!
df_gis = pd.merge(df_paleoz, gis, on='SitID')
df_new=pd.merge(df_gis, df_new, on='SitID')
df_new.head()

,SitID,UWI (AGS)_x,UWI_x,HorID_x,Pick_x,Quality_x,UWI (AGS)_y,UWI_y,HorID_y,Pick_y,Quality_y,lat,lng,UWI (AGS),UWI,HorID,Pick,Quality
0,102496,0674010812000,00/12-08-067-01W4/0,14000,561,1,0674010812000,00/12-08-067-01W4/0,13000,475,3,54.785907,-110.129320,0674010812000,00/12-08-067-01W4/0,13000,475,3
1,102497,0674020807000,00/07-08-067-02W4/0,14000,604.5,1,0674020807000,00/07-08-067-02W4/0,13000,515,3,54.782284,-110.269446,0674020807000,00/07-08-067-02W4/0,13000,515,3
2,102498,0674021109000,00/09-11-067-02W4/0,14000,564,1,0674021109000,00/09-11-067-02W4/0,13000,480,3,54.785892,-110.186851,0674021109000,00/09-11-067-02W4/0,13000,480,3
3,102500,0674022910000,00/10-29-067-02W4/0,14000,636.5,1,0674022910000,00/10-29-067-02W4/0,13000,549,3,54.829624,-110.269422,0674022910000,00/10-29-067-02W4/0,13000,549,3
4,102501,0674023406000,00/06-34-067-02W4/0,14000,613,1,0674023406000,00/06-34-067-02W4/0,13000,529,2,54.840471,-110.224832,0674023406000,00/06-34-067-02W4/0,13000,529,2


## Possible Things to Do with Lat / Long

### Features
1. Find average pick depth of N neighbor wells within a certain maximum distance
2. Return an array of pick depths at N neighbor wells within a certain maximum distance.
3. Return the average depth above or below another known pickin N nearest neighbor wells
3. Return an array of the depths above or below another known pickin N nearest neighbor wells
4. -
5. Return the amount of range in # 2
6. -
7. Return the amount of range in # 4
8. Return if a minimum number of wells are within a certain distance
9. Return the normalized hamming distance between a window around the pick in N nearst wells and window around a point in the well in question
10. Return something about the amount of variance or max/min gr or resititivity within window in question and around pick in neighboring well(s).
11. Find pick depth of nearest well 
12. Find distance above / below known pick of pick in question in nearest well
13. Find average value of GR, resititivity, DPHI, or ? in picks of nearby well 
14. Get array of average values of some well curve within a window of pick in question in K neighor wells.
15. *Return array of UWIs of K neighbor wells.*

###  Mapping
1. Show error on map
2. Show distribution of a feature on a map.
3. Identify where jumps in pick depth occur.
4. Identify where areas of greater variance in depth occur.
5. Identify where different unsupervized groupings occur.


## Possible Steps to Chain for some of the possible features
0. Save and access pickle of all the CSVs chained into single data frame
1. Return array of UWIs of K neighbor wells, place as list in data frame
2. Place nearest well UWI of each well into new column 
3. Place nearest well pick depths into new column
4. place average pick depths of the array of wells into new column
5. do same thing for nearest well and nearest K wells but for average Gr, DPHI, and Resitivity value if present.
6. Calculate nearest and average value for distance between pick in question and known pick.

In [6]:
position = df_new[['lat','lng']]

In [7]:
position

,lat,lng
0,54.785907,-110.129320
1,54.782284,-110.269446
2,54.785892,-110.186851
3,54.829624,-110.269422
4,54.840471,-110.224832
5,54.771449,-110.402983
6,54.785901,-110.422131
7,54.800533,-110.345762
8,54.829633,-110.396621
9,54.840441,-110.320301


In [8]:
tree = neighbors.KDTree(position, leaf_size=2) 

### Saving the tree of all the lat/lng points into pickle file named 'latlng_kd_tree_size2'

In [9]:
latlng_kd_tree_size2 = pickle.dumps(tree) 

### Making sure the kd_tree worked as indended

In [10]:
dist, ind = tree.query([position][0], k=5)  

In [11]:
len(dist)

2193

In [12]:
len(ind)

2193

In [13]:
ind[4:5]

array([[ 4,  3, 64, 63,  2]])

In [14]:
df_new[63:64]

,SitID,UWI (AGS)_x,UWI_x,HorID_x,Pick_x,Quality_x,UWI (AGS)_y,UWI_y,HorID_y,Pick_y,Quality_y,lat,lng,UWI (AGS),UWI,HorID,Pick,Quality
63,103699,0684020810000,00/10-08-068-02W4/0,14000,658.5,1,0684020810000,00/10-08-068-02W4/0,13000,572,1,54.873181,-110.269399,0684020810000,00/10-08-068-02W4/0,13000,572,1


In [15]:
df_new[4:5]

,SitID,UWI (AGS)_x,UWI_x,HorID_x,Pick_x,Quality_x,UWI (AGS)_y,UWI_y,HorID_y,Pick_y,Quality_y,lat,lng,UWI (AGS),UWI,HorID,Pick,Quality
4,102501,0674023406000,00/06-34-067-02W4/0,14000,613,1,0674023406000,00/06-34-067-02W4/0,13000,529,2,54.840471,-110.224832,0674023406000,00/06-34-067-02W4/0,13000,529,2


In [20]:
center = [54.840471, -110.269399]
zoom = 6


In [116]:
m = Map(default_tiles=TileLayer(opacity=1.0), center=center, zoom=zoom)
m.interact(zoom=(5,10,1))

mark = Marker(location=[54.873181, -110.269399])
mark.visible
m += mark



In [117]:
m

In [43]:

print(position[0:][0:1]['lat'])
type(position[0:][0:1]['lat'])

0    54.785907
Name: lat, dtype: float64


pandas.core.series.Series

In [46]:
print(position[0:][0:1]['lat'][0])
type(position[0:][0:1]['lat'][0])

54.785906999999995


numpy.float64

In [62]:
position

,lat,lng
0,54.785907,-110.129320
1,54.782284,-110.269446
2,54.785892,-110.186851
3,54.829624,-110.269422
4,54.840471,-110.224832
5,54.771449,-110.402983
6,54.785901,-110.422131
7,54.800533,-110.345762
8,54.829633,-110.396621
9,54.840441,-110.320301


In [52]:
position[0:]

,lat,lng
0,54.785907,-110.129320
1,54.782284,-110.269446
2,54.785892,-110.186851
3,54.829624,-110.269422
4,54.840471,-110.224832
5,54.771449,-110.402983
6,54.785901,-110.422131
7,54.800533,-110.345762
8,54.829633,-110.396621
9,54.840441,-110.320301


In [55]:
position[1:][0:1]

,lat,lng
1,54.782284,-110.269446


In [57]:
len(position[0:])

2193

In [118]:
print(position[0:][0:1]['lat'][0])

54.785906999999995


In [134]:
print(position[0:][0:1])

         lat        lng
0  54.785907 -110.12932


In [145]:
new_position = position.values.tolist()
print(new_position)

[[54.785906999999995, -110.12931999999999], [54.782284, -110.269446], [54.785892000000004, -110.186851], [54.829624, -110.26942199999999], [54.840470999999994, -110.224832], [54.771449, -110.402983], [54.785901, -110.42213100000001], [54.800532999999994, -110.34576200000001], [54.829632999999994, -110.396621], [54.840441000000006, -110.320301], [54.771451, -110.498414], [54.785895999999994, -110.599928], [54.814989000000004, -110.479147], [54.840481999999994, -110.549109], [54.775090000000006, -110.688675], [54.79695400000001, -110.70789599999999], [54.826012, -110.625318], [54.84411600000001, -110.701599], [54.775087, -110.77791], [54.796946999999996, -110.88586200000002], [54.811402, -110.80330900000001], [54.82247099999999, -110.879581], [54.771468000000006, -111.038407], [54.796937, -110.936727], [54.82246, -110.936727], [54.826041000000004, -111.025865], [54.771478, -111.13386000000001], [54.796927000000004, -111.089256], [54.829657999999995, -111.184726], [54.840503000000005, -11

###### note: adding all these markers to the map will take a while

In [146]:
circle = []
for row in new_position:
    c = Circle(location=list(row), radius=1000)
    print(c)
    circles.append(c)
    m.add_layer(c)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


...

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [12]:
#### 1. Find average pick depth of N neighbor wells within a certain maximum distance
#### takes in array or pandas df? returns dataframe?
def avgPickDepthForKneighborsWcertainDist(array,k_neighbors,max_distance):
    
    
    return uwi_array, distance_array
    

In [13]:
#### 1. Find average pick depth of N neighbor wells within a certain maximum distance
#### takes in array or pandas df? returns dataframe?
def avgPickDepthForKneighborsWcertainDist(array,k_neighbors,max_distance):
    return uwi_nearest, uwi_array, nearest_TopM_depth, avg_TopM_depth, nearest_BaseM_depth, avg_BaseM_depth
    

In [9]:
#### 2. Return an array of pick depths at N neighbor wells within a certain maximum distance.
#### takes in array or pandas df? returns dataframe?
def arrayOfPickDepthForKneighborsWcertainDist(array,k_neighbors,max_distance):
    return array_of_depths
    

In [10]:
#### 3. Return the average depth above or below another known pickin N nearest neighbor wells
#### takes in array or pandas df? returns dataframe?
def avgDepthAbvOrBelowKneighborsWcertainDist(array,k_neighbors,max_distance,aboveOrbelow,pickA,pickB):
    return avg_of_depthsAbvBelow

In [11]:
#### 4. Return the average depth above or below another known pickin N nearest neighbor wells
#### takes in array or pandas df? returns dataframe?
def arrayDepthAbvOrBelowKneighborsWcertainDist(array,k_neighbors,max_distance,aboveOrbelow,pickA,pickB):
    return array_of_depthsAbvBelow